# Old stuff

In [6]:
import spacy
spacy.load('en_core_web_md')
from epitator.annotator import AnnoDoc
from epitator.geoname_annotator import GeonameAnnotator
from epitator.resolved_keyword_annotator import ResolvedKeywordAnnotator
from epitator.count_annotator import CountAnnotator
from epitator.date_annotator import DateAnnotator
from boilerpipe.extract import Extractor
from itertools import groupby
import datetime
import sys
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm
import numpy as np
import epitator

In [9]:
from wiki_country_parser import get_wiki_countries_df

def match_country(country, look_up, translation):
    found = list(filter(lambda x: re.findall(country, x), look_up))
    if len(found) == 1:
        return translation[look_up.index(found[0])]
    elif len(found) > 1:
        # Check if identity in ambiguous e.g Niger --> (Niger, Nigeria)
        identical = [found_name for found_name in found if found_name == country]
        if len(identical) == 1:
            return translation[look_up.index(identical[0])]
        else:
            return [translation[look_up.index(similar)] for similar in found]


def translate(to_translate, look_up=wiki_countries_df):
    if isinstance(look_up, pd.DataFrame):
        wiki_countries_df = look_up
    else:
        wiki_countries_df = get_wiki_countries_df()

    continents = ["europa", "africa", "america", "australien", "asia"]
    state_name_de = wiki_countries_df["state_name_de"].tolist()
    full_state_name_de = wiki_countries_df["full_state_name_de"].tolist()
    translation = wiki_countries_df["translation_state_name"].tolist()
    match = None

    if isinstance(to_translate, str):
        match = match_country(to_translate, state_name_de, translation)
        if not match:
            match = match_country(to_translate, full_state_name_de, translation)
        elif not match:
            match = match_country(to_translate, translation, translation)
        # If still no match
        elif not match:
            did_u_mean = didyoumean.didYouMean(to_translate, state_name_de)
            if did_u_mean and (did_u_mean not in continents):
                match = translate(did_u_mean)
            else:
                match = to_translate
    elif isinstance(to_translate, list):
        print("no")
        match = [translate(country) for country in to_translate]
    return match

example_to_translate = ["Deutschland", "Delaware", ["Kongo", "China"], "Niger"]
# flatten_list([translate(example) for example in example_to_translate])

In [4]:
wiki_countries_df = get_wiki_countries_df()
continents = ["europa", "africa", "america", "australien", "asia"]
state_name_de = wiki_countries_df["state_name_de"].tolist()
full_state_name_de = wiki_countries_df["full_state_name_de"].tolist()
translation = wiki_countries_df["translation_state_name"].tolist()

This is scraped unexpectedly. Don't execute to often


In [12]:
match_country("Kongo", state_name_de,translation)

here


['Congo, Democratic Republic of the (Kinshasa)',
 'Congo, Republic of (Brazzaville)']

In [16]:
list(filter(lambda x: re.findall("Congo", x), translation))

['Congo, Democratic Republic of the (Kinshasa)',
 'Congo, Republic of (Brazzaville)']

In [ ]:
lol = True
while(lol):
    print("j")
    lol = False
    print("n")

In [ ]:
match_country("Kongo", state_name_de,translation)
list(filter(lambda x: re.findall("Kongo", x), state_name_de))
translation[state_name_de.index('Kongo, Demokratische Republik')]

In [147]:
a = list(filter(lambda x: re.findall("Niger", x), state_name_de))
identical = [found_name for found_name in a if "Niger" == state_name_de]

In [150]:
a

['Niger', 'Nigeria']

In [107]:
from wiki_country_parser import get_wiki_countries_df
get_wiki_countries_df()

,state_name_de,full_state_name_de,capital_de,translation_state_name,wiki_abbreviations,inoff_abbreviations
0,Erde,—,—,Earth,—,-
1,Europäische Union,—,Brüssel,European Union,—,[EU]
2,Union Südamerikanischer Nationen,—,Quito,Union of South American Nations,—,"[USN, USAN]"
3,Afrikanische Union,—,Addis Abeba,African Union,—,[AU]
4,Verband Südostasiatischer Nationen,—,Jakarta,Association of Southeast Asian Nations,—,"[VSN, ASAN]"
5,Organisation Amerikanischer Staaten,—,"Washington, D.C.",Organization of American States,—,[OAS]
6,Abchasien,Republik Abchasien,Sochumi,Abkhazia,—,[RA]
7,Afghanistan,Islamische Republik Afghanistan,Kabul,Afghanistan,"[AFG, AF]",[IRA]
8,Ägypten,Arabische Republik Ägypten,Kairo,Egypt,"[EGY, EG]",[ARÄ]
9,Albanien,Republik Albanien,Tirana,Albania,"[ALB, AL]",[RA]


## Read in Ereignisdatenbank

In [ ]:
ereignisdatenbank = pd.read_csv("Ereignisse_utf8.csv",sep=";")

In [ ]:
ereignisdatenbank.columns = list(map(lambda x:x.strip(" "),ereignisdatenbank.columns))

In [ ]:
countries_not_null = ereignisdatenbank[pd.notnull(ereignisdatenbank["Ausgangs- bzw. Ausbruchsland"])]

In [ ]:
countries = countries_not_null["Ausgangs- bzw. Ausbruchsland"].copy(deep=True)
countries = list(map(lambda x:x.strip(" "),countries))

# Let the testing begin


## Ontology/Comparison (Transformed to .py until here)

In [ ]:
from epitator.annotator import AnnoDoc
from epitator.geoname_annotator import location_contains
doc = AnnoDoc("I live in Munic!")
doc.add_tiers(GeonameAnnotator())
annotations = doc.tiers["geonames"]
geoname = annotations[0]
geoname.to_dict()

In [ ]:
## SIMPLE TRANSLATION. FAST BUT DOES NOT TRANSLATE LISTS OF LISTS
# # Translate German entries of Ereignisdatenbank to English. Might be inefficient since I go through the wiki list
# # entirely which is longer then the list of countries to translate
# translated_ereignisdatenbank_countries = [(entry,wikipedia_country_list["translation_state_name"].tolist()[indx])\
#                                           for indx,entry \
#                                           in enumerate(wikipedia_country_list["state_name_de"].tolist())\
#                                           if entry in countries_unique]

In [ ]:
wikipedia_country_list = get_wiki_countries_df()

In [ ]:
state_name_de = wikipedia_country_list["state_name_de"].tolist()
list(filter(lambda x: re.findall("Italien",x),state_name_de))

In [ ]:
from didyoumean import didyoumean

def translate(countries_unique):
    """Translate German entries of Ereignisdatenbank to English an returns tuple 
    of German word and English translation(s if ambigious)""" 

    continents = ["europa","africa","america","australien","asia"]
    translated_ereignisdatenbank_countries = []
    state_name_de = wikipedia_country_list["state_name_de"].tolist()
    full_state_name_de = wikipedia_country_list["full_state_name_de"].tolist()
    translation = wikipedia_country_list["translation_state_name"].tolist()
    
    if type(countries_unique) == str:
        countries_unique = [countries_unique]
    
    for entry in countries_unique:
        if type(entry) == str:
            sucessfull_search = list(filter(lambda x: re.findall(entry,x),state_name_de))
            if sucessfull_search:
                found = [translation[state_name_de.index(entry)] for entry in sucessfull_search]
                if len(found) == 1:
                    translated_ereignisdatenbank_countries.append((entry,found[0]))
                else:
                    
                    # Check for idendity in not ambigious case otherwise e.g Niger --> (Niger, Nigeria)
                    identical = [found_ent for found_ent in found if entry == found_ent]
                    if identical:
                        translated_ereignisdatenbank_countries.append((entry,identical[0]))
                    else:
                        translated_ereignisdatenbank_countries.append((entry,found))
            else:
                
                # If entry not in state_name_de, search in full_state_name_de
                sucessfull_search = list(filter(lambda x: re.findall(entry,x),full_state_name_de))
                if sucessfull_search:
                    found = [translation[full_state_name_de.index(entry)] for entry in sucessfull_search]
                    if len(found) == 1:
                        translated_ereignisdatenbank_countries.append((entry,found[0]))
                    else:
                        translated_ereignisdatenbank_countries.append((entry,found))
                else:
                    sucessfull_search_en = list(filter(lambda x: re.findall(entry,x),translation))
                    if sucessfull_search_en:
                        found = [state_name_de[translation.index(entry)] for entry in sucessfull_search_en]
                        if len(found) == 1:
                            translated_ereignisdatenbank_countries.append((found[0],entry))
                        else:
                            translated_ereignisdatenbank_countries.append((found,entry))
                    else:
                        
                        # If there was not match at all, check for spelling mistakes
                        did_u_mean = didyoumean.didYouMean(entry,state_name_de)
                        
                        """Exlude words with continent names since there are countries with a continent name
                        but there are also entries in the Ereignisdatenbank that mean the whole country. They
                        are not matched (e.g. Nordafrika) and there is must not be a match, otherwise 
                        didYouMean would falsly return Südafrika.
                        """
                        if did_u_mean and (did_u_mean not in continents):
                            translated_ereignisdatenbank_countries.append(translate(did_u_mean))
                        else:
                            translated_ereignisdatenbank_countries.append(entry)
                                                            
        elif type(entry) == list:
            translated_ereignisdatenbank_countries.append(translate(entry))
        else:
            translated_ereignisdatenbank_countries.append(entry)
    return translated_ereignisdatenbank_countries

In [ ]:
ereignisdatenbank["Ausgangs- bzw. Ausbruchsland "].tolist()

In [ ]:
example_to_translate = ["Deutschland","Delaware",["Kongo","China"],"Niger"]
expected_result_translate = [('Deutschland', 'Germany'),
                             'Delaware',
                             [('Kongo',
                               ['Congo, Democratic Republic of the (Kinshasa)','Congo, Republic of (Brazzaville)']),
                              ('China','China')],
                            ('Niger', 'Niger')]
if deep_eq(translate(example_to_translate),expected_result_translate):
    print("Test succesfull")
else:
    print("Test failed")

In [ ]:
translated = translate(translate_abbreviation(clean_entries(countries)))
translated

In [ ]:
flattened = [entry if type(sublist) == list else sublist for sublist in translated for entry in sublist ]
countries_not_translated = set([entry for entry in flattened if type(entry) == str])
countries_not_translated